Import relevant

Link: https://colab.research.google.com/drive/1H4ifh5RXESMoVWBgjtCOvT-pBIbJRPaz?usp=sharing

In [ ]:
#Import relevant
import qiskit as qc
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer
from qiskit.quantum_info import Statevector
import numpy as np

Now for the function:

In [ ]:
def is_rectangle(A: int, B: int, C: int, D: int) -> int:
    #Define relevant quantum circuits and registers.
    #We want to cover each of the possible configurations for the sides, i.e. AB, AC, AD, BC, BD and CD
    #This requires 6 Circuits.
    qr = QuantumRegister(3)
    cr = ClassicalRegister(2)
    
    qcab = QuantumCircuit(qr, cr)
    qcac = QuantumCircuit(qr, cr)
    qcad = QuantumCircuit(qr, cr)
    qcbc = QuantumCircuit(qr, cr)
    qcbd = QuantumCircuit(qr, cr)
    qccd = QuantumCircuit(qr, cr)
    
    #Define each of the first qubit states as |0>
    #Why we are doing this will be apparent in the swap circuit steps
    qcab.x(0)
    qcac.x(0)
    qcad.x(0)
    qcbc.x(0)
    qcbd.x(0)
    qccd.x(0)
    
    sides = [A,B,C,D] #A,B,C,D
    
    #We want these A, B, C, D values to be normalized to use the arcsin function.
    if sides[0] == sides[1] == sides[2] == sides[3] :
        normalizedsides=[1,1,1,1]
    else:
        normalizedsides = (sides-np.min(sides))/(np.max(sides)-np.min(sides))
    #Now we have a normalized list of relative values between [0,1]
    
    #Now we assign this normalized list to representative qubit representations.
    #This is done for all the circuits, leaving the first qubit as the |0> state.
    #AB
    qcab.ry(2*np.arcsin(normalizedsides[0]),1)
    qcab.ry(2*np.arcsin(normalizedsides[1]),2)
    #AC
    qcac.ry(2*np.arcsin(normalizedsides[0]),1)
    qcac.ry(2*np.arcsin(normalizedsides[2]),2)
    #AD
    qcad.ry(2*np.arcsin(normalizedsides[0]),1)
    qcad.ry(2*np.arcsin(normalizedsides[3]),2)
    #BC
    qcbc.ry(2*np.arcsin(normalizedsides[1]),1)
    qcbc.ry(2*np.arcsin(normalizedsides[2]),2)
    #BD
    qcbd.ry(2*np.arcsin(normalizedsides[1]),1)
    qcbd.ry(2*np.arcsin(normalizedsides[3]),2)
    #CD
    qccd.ry(2*np.arcsin(normalizedsides[2]),1)
    qccd.ry(2*np.arcsin(normalizedsides[3]),2)
    
    #Perform needed swaps
    # The swap circuit is quite simple:
    #
    # |0> --- H --- o --- H --- Measure
    #               |
    # a1  --------- x ---------
    #               |
    # a2  ----------x ---------
    #AB swap
    qcab.h(0)
    qcab.cswap(0, 1, 2)
    qcab.h(0)
    #AC swap
    qcac.h(0)
    qcac.cswap(0, 1, 2)
    qcac.h(0)
    #AD swap
    qcad.h(0)
    qcad.cswap(0, 1, 2)
    qcad.h(0)
    #BC swap
    qcbc.h(0)
    qcbc.cswap(0, 1, 2)
    qcbc.h(0)
    #BD swap
    qcbd.h(0)
    qcbd.cswap(0, 1, 2)
    qcbd.h(0)
    #CD swap
    qccd.h(0)
    qccd.cswap(0, 1, 2)
    qccd.h(0)

    #Now we convert to a statevector-we want to have probabilities 
    #For this we convert our circuits to relative amplitude values
    #We take these relative amplitude values for our probability.
    psiab=Statevector.from_instruction(qcab)
    psiac=Statevector.from_instruction(qcac)
    psiad=Statevector.from_instruction(qcad)
    psibc=Statevector.from_instruction(qcbc)
    psibd=Statevector.from_instruction(qcbd)
    psicd=Statevector.from_instruction(qccd)
    
    #Take probabilities
    psiabp=psiab.probabilities()
    psiacp=psiac.probabilities()
    psiadp=psiad.probabilities()
    psibcp=psibc.probabilities()
    psibdp=psibd.probabilities()
    psicdp=psicd.probabilities()
    

    #The swap circuit returns the state |0> with 1.0 probability if the two state inputs are equal.
    #Therefore, we now want to iterate through all 8 entries (2^3 possible values for the qubits).
    #If we see a one value in any array, add one to counter.
    #Iff counter == 2, then we have a valid rectangle.
    
    rect=0
    
    maxval=[]
    
    maxval.append(np.max(psiacp))
    maxval.append(np.max(psiadp))
    maxval.append(np.max(psibcp))
    maxval.append(np.max(psibdp))
    maxval.append(np.max(psicdp))
    maxval.append(np.max(psiabp))
    
    for element in maxval:
        if element >= 0.99995:
            rect +=1
        
    
    #now if rect == 2, return 1.
    #There are rounding errors associated with this schema.
    #Noticably, the first AB element typically returns a value
    #very very close to 1, but not 1, around 0.9999999998.
    #This is why we needed an element >=0.99995 clause above.
    #This means that the code does break down when elements are very close to each other
    #take is_rectangle(1,1,5,5.00001). This would likely return 1.
    if (rect == 2):
        return 1
    else:
        return 0
    
            